# **3d Solids**

## Mesh

In [1]:
from polymesh.grid import gridH8 as grid
import numpy as np

size = Lx, Ly, Lz = (20, 4, 4)
shape = nx, ny, nz = (40, 10, 10)

gridparams = {
    'size': size,
    'shape': shape,
    'origo': (0, 0, 0),
    'start': 0
}

coords, topo = grid(**gridparams)

## Material

In [2]:
E = 12000.
nu = 0.2
A = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E / (1-nu**2))


## Boundary Conditions

In [3]:
# fix points at x==0
cond = coords[:, 0] <= 0.001
ebcinds = np.where(cond)[0]
fixity = np.zeros((coords.shape[0], 6), dtype=bool)
fixity[ebcinds, :] = True
fixity[:, 3:] = True

# unit vertical load at (Lx, Ly)
F = 10
cond = (coords[:, 0] > (Lx-(1e-12))) & \
    (coords[:, 1] > (Ly-(1e-12))) & \
        (coords[:, 2] > (Lz-(1e-12)))
nbcinds = np.where(cond)[0]
loads = np.zeros((coords.shape[0], 6))
loads[nbcinds, 2] = -F


## Assembly and Solution

In [4]:
from sigmaepsilon.solid import Structure, PointData, FemMesh
from polymesh.space import StandardFrame
from sigmaepsilon.solid.fem.cells import H8 as CellData
from neumann.array import repeat

GlobalFrame = StandardFrame(dim=3)

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = repeat(GlobalFrame.show(), topo.shape[0])
cd = CellData(topo=topo, frames=frames)

# set up mesh and structure
mesh = FemMesh(pd, cd, model=A, frame=GlobalFrame)
structure = Structure(mesh=mesh)

In [5]:
structure.linsolve()
dofsol = structure.nodal_dof_solution()
structure.mesh.pointdata['x'] = coords + dofsol[:, :3]